In [ ]:
# Download necessary libraries
!pip install pyspark

In [ ]:
# Import necessary libraries
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import *

In [ ]:
# Create a Spark session
spark = SparkSession.builder.appName("DataAnalysis").getOrCreate()

# Load the dataset
df_matches = spark.read.format('csv').options(header = 'True').load('/content/drive/MyDrive/Colab Notebooks/data/football_matches.csv')

# Preview of the data
df_matches.toPandas()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1,D2,2009,2010-04-04,Oberhausen,Kaiserslautern,2,1,H
1,2,D2,2009,2009-11-01,Munich 1860,Kaiserslautern,0,1,A
2,3,D2,2009,2009-10-04,Frankfurt FSV,Kaiserslautern,1,1,D
3,4,D2,2009,2010-02-21,Frankfurt FSV,Karlsruhe,2,1,H
4,5,D2,2009,2009-12-06,Ahlen,Karlsruhe,1,3,A
...,...,...,...,...,...,...,...,...,...
24620,46770,E0,2016,2017-05-21,Liverpool,Middlesbrough,3,0,H
24621,46771,E0,2016,2017-05-21,Man United,Crystal Palace,2,0,H
24622,46772,E0,2016,2017-05-21,Southampton,Stoke,0,1,A
24623,46773,E0,2016,2017-05-21,Swansea,West Brom,2,1,H


**Meaning of the features :**

- Match_ID : Unique identifier for each match
- Div : Championship level (D1 = first level, D2 = second level, E0 = third level)
- Season : Starting year of the season in which the match took place
- Date : Date of the match
- HomeTeam : Team playing in its stadium
- AwayTeam : Team playing in the opposing team's stadium
- FTHG : Number of goals of Home Team
- FTAG : Number of goals of Away Team
- FTR : Final Result

In [ ]:
# Rename column names
df_matches = df_matches.selectExpr(
    "*",
    "`FTHG` AS `HomeTeamGoals`",
    "`FTAG` AS `AwayTeamGoals`",
    "`FTR` AS `FinalResult`"
)

# Preview of the data
df_matches.toPandas()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HomeTeamGoals,AwayTeamGoals,FinalResult
0,1,D2,2009,2010-04-04,Oberhausen,Kaiserslautern,2,1,H,2,1,H
1,2,D2,2009,2009-11-01,Munich 1860,Kaiserslautern,0,1,A,0,1,A
2,3,D2,2009,2009-10-04,Frankfurt FSV,Kaiserslautern,1,1,D,1,1,D
3,4,D2,2009,2010-02-21,Frankfurt FSV,Karlsruhe,2,1,H,2,1,H
4,5,D2,2009,2009-12-06,Ahlen,Karlsruhe,1,3,A,1,3,A
...,...,...,...,...,...,...,...,...,...,...,...,...
24620,46770,E0,2016,2017-05-21,Liverpool,Middlesbrough,3,0,H,3,0,H
24621,46771,E0,2016,2017-05-21,Man United,Crystal Palace,2,0,H,2,0,H
24622,46772,E0,2016,2017-05-21,Southampton,Stoke,0,1,A,0,1,A
24623,46773,E0,2016,2017-05-21,Swansea,West Brom,2,1,H,2,1,H


In [ ]:
# Delete FTHG, FTAG and FTR columns
df_matches = df_matches.drop("FTHG", "FTAG", "FTR")

# Preview of the data
df_matches.toPandas()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,HomeTeamGoals,AwayTeamGoals,FinalResult
0,1,D2,2009,2010-04-04,Oberhausen,Kaiserslautern,2,1,H
1,2,D2,2009,2009-11-01,Munich 1860,Kaiserslautern,0,1,A
2,3,D2,2009,2009-10-04,Frankfurt FSV,Kaiserslautern,1,1,D
3,4,D2,2009,2010-02-21,Frankfurt FSV,Karlsruhe,2,1,H
4,5,D2,2009,2009-12-06,Ahlen,Karlsruhe,1,3,A
...,...,...,...,...,...,...,...,...,...
24620,46770,E0,2016,2017-05-21,Liverpool,Middlesbrough,3,0,H
24621,46771,E0,2016,2017-05-21,Man United,Crystal Palace,2,0,H
24622,46772,E0,2016,2017-05-21,Southampton,Stoke,0,1,A
24623,46773,E0,2016,2017-05-21,Swansea,West Brom,2,1,H


In [ ]:
# Preview of the data in Pandas DataFrame format
df_matches.toPandas()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,HomeTeamGoals,AwayTeamGoals,FinalResult
0,1,D2,2009,2010-04-04,Oberhausen,Kaiserslautern,2,1,H
1,2,D2,2009,2009-11-01,Munich 1860,Kaiserslautern,0,1,A
2,3,D2,2009,2009-10-04,Frankfurt FSV,Kaiserslautern,1,1,D
3,4,D2,2009,2010-02-21,Frankfurt FSV,Karlsruhe,2,1,H
4,5,D2,2009,2009-12-06,Ahlen,Karlsruhe,1,3,A
...,...,...,...,...,...,...,...,...,...
24620,46770,E0,2016,2017-05-21,Liverpool,Middlesbrough,3,0,H
24621,46771,E0,2016,2017-05-21,Man United,Crystal Palace,2,0,H
24622,46772,E0,2016,2017-05-21,Southampton,Stoke,0,1,A
24623,46773,E0,2016,2017-05-21,Swansea,West Brom,2,1,H


In [ ]:
# Create binary columns for wins, losses and draws
df_matches = df_matches.withColumn("HomeTeamWin", when(col("FinalResult") == "H", 1).otherwise(0)) \
                        .withColumn("AwayTeamWin", when(col("FinalResult") == "A", 1).otherwise(0)) \
                        .withColumn("GameTie", when(col("FinalResult") == "D", 1).otherwise(0))

# Preview of the data
df_matches.toPandas()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,HomeTeamGoals,AwayTeamGoals,FinalResult,HomeTeamWin,AwayTeamWin,GameTie
0,1,D2,2009,2010-04-04,Oberhausen,Kaiserslautern,2,1,H,1,0,0
1,2,D2,2009,2009-11-01,Munich 1860,Kaiserslautern,0,1,A,0,1,0
2,3,D2,2009,2009-10-04,Frankfurt FSV,Kaiserslautern,1,1,D,0,0,1
3,4,D2,2009,2010-02-21,Frankfurt FSV,Karlsruhe,2,1,H,1,0,0
4,5,D2,2009,2009-12-06,Ahlen,Karlsruhe,1,3,A,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
24620,46770,E0,2016,2017-05-21,Liverpool,Middlesbrough,3,0,H,1,0,0
24621,46771,E0,2016,2017-05-21,Man United,Crystal Palace,2,0,H,1,0,0
24622,46772,E0,2016,2017-05-21,Southampton,Stoke,0,1,A,0,1,0
24623,46773,E0,2016,2017-05-21,Swansea,West Brom,2,1,H,1,0,0


In [ ]:
# Create a dataframe containing only Bundesliga matches
df_bundesliga = df_matches.filter((col('Div') == 'D1') &
                            (col('Season') >= 2000) &
                            (col('Season') <= 2015))
# Preview of the data
df_bundesliga.toPandas()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,HomeTeamGoals,AwayTeamGoals,FinalResult,HomeTeamWin,AwayTeamWin,GameTie
0,21,D1,2009,2010-02-06,Bochum,Leverkusen,1,1,D,0,0,1
1,22,D1,2009,2009-11-22,Bayern Munich,Leverkusen,1,1,D,0,0,1
2,23,D1,2009,2010-05-08,M'gladbach,Leverkusen,1,1,D,0,0,1
3,24,D1,2009,2009-08-08,Mainz,Leverkusen,2,2,D,0,0,1
4,25,D1,2009,2009-10-17,Hamburg,Leverkusen,0,0,D,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
4891,37179,D1,2004,2005-05-21,Hertha,Hannover,0,0,D,0,0,1
4892,37180,D1,2004,2005-05-21,Kaiserslautern,Werder Bremen,1,2,A,0,1,0
4893,37181,D1,2004,2005-05-21,Leverkusen,M'gladbach,5,1,H,1,0,0
4894,37182,D1,2004,2005-05-21,Nurnberg,Mainz,1,2,A,0,1,0


In [ ]:
# Create a dataframe containing statistics on home matches
df_home_matches = df_bundesliga.groupby('Season', 'HomeTeam') \
        .agg(sum('HomeTeamWin').alias('TotalHomeWin'),
             sum('AwayTeamWin').alias('TotalHomeLoss'),
             sum('GameTie').alias('TotalHomeTie'),
             sum('HomeTeamGoals').alias('HomeScoredGoals'),
             sum('AwayTeamGoals').alias('HomeAgainstGoals')) \
        .withColumnRenamed('HomeTeam', 'Team')

# Preview of the data
df_home_matches.toPandas()

,Season,Team,TotalHomeWin,TotalHomeLoss,TotalHomeTie,HomeScoredGoals,HomeAgainstGoals
0,2011,Hamburg,3,7,7,19.0,29.0
1,2005,Kaiserslautern,5,7,5,26.0,33.0
2,2006,Cottbus,6,6,5,21.0,22.0
3,2001,St Pauli,4,9,4,19.0,28.0
4,2005,Mainz,6,4,7,31.0,23.0
...,...,...,...,...,...,...,...
283,2004,Bielefeld,7,7,3,21.0,21.0
284,2004,Werder Bremen,9,4,4,33.0,15.0
285,2013,Augsburg,9,5,3,27.0,22.0
286,2004,Stuttgart,12,3,2,34.0,15.0


In [ ]:
# Create a dataframe containing statistics on away matches
df_away_matches = df_bundesliga.groupby('Season', 'AwayTeam') \
        .agg(sum('AwayTeamWin').alias('TotalAwayWin'),
             sum('HomeTeamWin').alias('TotalAwayLoss'),
             sum('GameTie').alias('TotalAwayTie'),
             sum('AwayTeamGoals').alias('AwayScoredGoals'),
             sum('HomeTeamGoals').alias('AwayAgainstGoals')) \
        .withColumnRenamed('AwayTeam', 'Team')

# Preview of the data
df_away_matches.toPandas()

,Season,Team,TotalAwayWin,TotalAwayLoss,TotalAwayTie,AwayScoredGoals,AwayAgainstGoals
0,2011,Hamburg,5,7,5,16.0,28.0
1,2005,Kaiserslautern,3,10,4,21.0,38.0
2,2006,Cottbus,5,9,3,17.0,27.0
3,2001,St Pauli,0,11,6,18.0,42.0
4,2005,Mainz,3,10,4,15.0,24.0
...,...,...,...,...,...,...,...
283,2004,Bielefeld,4,9,4,16.0,28.0
284,2004,Werder Bremen,9,7,1,35.0,22.0
285,2013,Augsburg,6,7,4,20.0,25.0
286,2004,Stuttgart,5,7,5,20.0,25.0


In [ ]:
# Join home and away data
df_merged = df_home_matches.join(df_away_matches, ['Team', 'Season'], 'inner')

# Preview of the data
df_merged.toPandas()

,Team,Season,TotalHomeWin,TotalHomeLoss,TotalHomeTie,HomeScoredGoals,HomeAgainstGoals,TotalAwayWin,TotalAwayLoss,TotalAwayTie,AwayScoredGoals,AwayAgainstGoals
0,Hamburg,2011,3,7,7,19.0,29.0,5,7,5,16.0,28.0
1,Kaiserslautern,2005,5,7,5,26.0,33.0,3,10,4,21.0,38.0
2,Cottbus,2006,6,6,5,21.0,22.0,5,9,3,17.0,27.0
3,St Pauli,2001,4,9,4,19.0,28.0,0,11,6,18.0,42.0
4,Mainz,2005,6,4,7,31.0,23.0,3,10,4,15.0,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...
283,Bielefeld,2004,7,7,3,21.0,21.0,4,9,4,16.0,28.0
284,Werder Bremen,2004,9,4,4,33.0,15.0,9,7,1,35.0,22.0
285,Augsburg,2013,9,5,3,27.0,22.0,6,7,4,20.0,25.0
286,Stuttgart,2004,12,3,2,34.0,15.0,5,7,5,20.0,25.0


In [ ]:
# Create columns for total scores and results
df_totals = df_merged.withColumn('GoalsScored', col('HomeScoredGoals') + col('AwayScoredGoals')) \
                      .withColumn('GoalsAgainst', col('HomeAgainstGoals') + col('AwayAgainstGoals')) \
                      .withColumn('Win', col('TotalHomeWin') + col('TotalAwayWin')) \
                      .withColumn('Loss', col('TotalHomeLoss') + col('TotalAwayLoss')) \
                      .withColumn('Tie', col('TotalHomeTie') + col('TotalAwayTie'))

# Preview of the data
df_totals.toPandas()

,Team,Season,TotalHomeWin,TotalHomeLoss,TotalHomeTie,HomeScoredGoals,HomeAgainstGoals,TotalAwayWin,TotalAwayLoss,TotalAwayTie,AwayScoredGoals,AwayAgainstGoals,GoalsScored,GoalsAgainst,Win,Loss,Tie
0,Hamburg,2011,3,7,7,19.0,29.0,5,7,5,16.0,28.0,35.0,57.0,8,14,12
1,Kaiserslautern,2005,5,7,5,26.0,33.0,3,10,4,21.0,38.0,47.0,71.0,8,17,9
2,Cottbus,2006,6,6,5,21.0,22.0,5,9,3,17.0,27.0,38.0,49.0,11,15,8
3,St Pauli,2001,4,9,4,19.0,28.0,0,11,6,18.0,42.0,37.0,70.0,4,20,10
4,Mainz,2005,6,4,7,31.0,23.0,3,10,4,15.0,24.0,46.0,47.0,9,14,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,Bielefeld,2004,7,7,3,21.0,21.0,4,9,4,16.0,28.0,37.0,49.0,11,16,7
284,Werder Bremen,2004,9,4,4,33.0,15.0,9,7,1,35.0,22.0,68.0,37.0,18,11,5
285,Augsburg,2013,9,5,3,27.0,22.0,6,7,4,20.0,25.0,47.0,47.0,15,12,7
286,Stuttgart,2004,12,3,2,34.0,15.0,5,7,5,20.0,25.0,54.0,40.0,17,10,7


In [ ]:
# Unnecessary columns
cols_to_drop = ['HomeScoredGoals', 'AwayScoredGoals', 'HomeAgainstGoals', 'AwayAgainstGoals', 'TotalHomeWin',
                'TotalAwayWin', 'TotalHomeLoss', 'TotalAwayLoss', 'TotalHomeTie', 'TotalAwayTie']

# Drop unnecessary columns
df_cleaned = df_totals.drop(*cols_to_drop)

# Preview of the data
df_cleaned.toPandas()

,Team,Season,GoalsScored,GoalsAgainst,Win,Loss,Tie
0,Hamburg,2011,35.0,57.0,8,14,12
1,Kaiserslautern,2005,47.0,71.0,8,17,9
2,Cottbus,2006,38.0,49.0,11,15,8
3,St Pauli,2001,37.0,70.0,4,20,10
4,Mainz,2005,46.0,47.0,9,14,11
...,...,...,...,...,...,...,...
283,Bielefeld,2004,37.0,49.0,11,16,7
284,Werder Bremen,2004,68.0,37.0,18,11,5
285,Augsburg,2013,47.0,47.0,15,12,7
286,Stuttgart,2004,54.0,40.0,17,10,7


In [ ]:
# Create additional columns
df_processed = df_cleaned.withColumn('GoalDifferentials', col('GoalsScored') - col('GoalsAgainst')) \
                          .withColumn('WinPercentage', round((100 * col('Win') / (col('Win') + col('Loss') + col('Tie'))), 2))

# Preview of the data
df_processed.toPandas()

,Team,Season,GoalsScored,GoalsAgainst,Win,Loss,Tie,GoalDifferentials,WinPercentage
0,Hamburg,2011,35.0,57.0,8,14,12,-22.0,23.53
1,Kaiserslautern,2005,47.0,71.0,8,17,9,-24.0,23.53
2,Cottbus,2006,38.0,49.0,11,15,8,-11.0,32.35
3,St Pauli,2001,37.0,70.0,4,20,10,-33.0,11.76
4,Mainz,2005,46.0,47.0,9,14,11,-1.0,26.47
...,...,...,...,...,...,...,...,...,...
283,Bielefeld,2004,37.0,49.0,11,16,7,-12.0,32.35
284,Werder Bremen,2004,68.0,37.0,18,11,5,31.0,52.94
285,Augsburg,2013,47.0,47.0,15,12,7,0.0,44.12
286,Stuttgart,2004,54.0,40.0,17,10,7,14.0,50.00


In [ ]:
# Set window for partition
window_partition = Window.partitionBy('Season').orderBy(col('WinPercentage').desc(), col('GoalDifferentials').desc())

# Rank teams by season
df_ranked = df_processed.withColumn('TeamPosition', rank().over(window_partition))

# Preview of the data
df_ranked.toPandas()

,Team,Season,GoalsScored,GoalsAgainst,Win,Loss,Tie,GoalDifferentials,WinPercentage,TeamPosition
0,Bayern Munich,2000,62.0,37.0,19,9,6,25.0,55.88,1
1,Schalke 04,2000,65.0,35.0,18,8,8,30.0,52.94,2
2,Hertha,2000,58.0,52.0,18,14,2,6.0,52.94,3
3,Leverkusen,2000,54.0,40.0,17,11,6,14.0,50.00,4
4,Dortmund,2000,62.0,42.0,16,8,10,20.0,47.06,5
...,...,...,...,...,...,...,...,...,...,...
283,Hoffenheim,2015,39.0,54.0,9,15,10,-15.0,26.47,14
284,Darmstadt,2015,38.0,53.0,9,14,11,-15.0,26.47,14
285,Ein Frankfurt,2015,34.0,52.0,9,16,9,-18.0,26.47,16
286,Stuttgart,2015,50.0,75.0,9,19,6,-25.0,26.47,17


In [ ]:
# Filter top teams
df_top_teams = df_ranked.filter(col('TeamPosition') == 1)
df_top_teams = df_top_teams.drop('TeamPosition')

# Preview of the data
df_top_teams.toPandas()

,Team,Season,GoalsScored,GoalsAgainst,Win,Loss,Tie,GoalDifferentials,WinPercentage
0,Bayern Munich,2000,62.0,37.0,19,9,6,25.0,55.88
1,Leverkusen,2001,77.0,38.0,21,7,6,39.0,61.76
2,Bayern Munich,2002,70.0,25.0,23,5,6,45.0,67.65
3,Werder Bremen,2003,79.0,38.0,22,4,8,41.0,64.71
4,Bayern Munich,2004,75.0,33.0,24,5,5,42.0,70.59
5,Bayern Munich,2005,67.0,32.0,22,3,9,35.0,64.71
6,Stuttgart,2006,61.0,37.0,21,6,7,24.0,61.76
7,Bayern Munich,2007,68.0,21.0,22,2,10,47.0,64.71
8,Wolfsburg,2008,80.0,41.0,21,7,6,39.0,61.76
9,Bayern Munich,2009,72.0,31.0,20,4,10,41.0,58.82
